In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Load dataset
df = pd.read_csv('nrmal.csv')
questions = df['question'].tolist()
answers = df['answer'].tolist()

'''# Sample dataset of questions and answers
questions = [
    "What is your name?",
    "How are you?",
    "What do you do?",
    "Tell me a joke.",
    "What is the weather like?"
]

answers = [
    "I am a chatbot.",
    "I am doing great, thank you!",
    "I am here to chat with you.",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "I don't know, what's your favorite weather?"
]'''

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

# Convert texts to sequences
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Pad sequences
max_len = max(max(len(seq) for seq in question_sequences),max(len(seq) for seq in question_sequences))
question_padded = pad_sequences(question_sequences, maxlen=max_len, padding='post')
answer_padded = pad_sequences(answer_sequences, maxlen=max_len, padding='post')

# Prepare input and output
X = question_padded
y = answer_padded

# Model parameters
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_dim = 128
rnn_units = 32
num_heads=8

# Build the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    tf.keras.layers.LSTM(vocab_size, return_sequences=True),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])



'''# Transformer model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    
    # Transformer layer using multi-head attention
    tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim),
    
    # Add a feed-forward network after attention
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    
    # Apply a normalization layer
    tf.keras.layers.LayerNormalization(),
    
    # Output layer for final prediction
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])'''

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, np.expand_dims(y, -1), epochs=200, verbose=1)

# Function to predict the answer based on the question
def predict_answer(question):
    question_seq = tokenizer.texts_to_sequences([question])
    question_padded = pad_sequences(question_seq, maxlen=max_len, padding='post')
    prediction = model.predict(question_padded)
    predicted_word_index = np.argmax(prediction, axis=-1)[0]
    predicted_words = [tokenizer.index_word[i] for i in predicted_word_index if i > 0]
    return ' '.join(predicted_words)

# Test the model with a sample question
test_question = "what is your name"
response = predict_answer(test_question)
print(f"Response: {response}")

Epoch 1/200


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0800 - loss: 3.4005
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 3.3805
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7400 - loss: 3.3602
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7700 - loss: 3.3392
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.7900 - loss: 3.3169
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7900 - loss: 3.2931
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7900 - loss: 3.2674
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7700 - loss: 3.2394
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7700 - loss: 3.2087
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7700 - loss: 3.1750
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7700 - loss: 3.1379
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7300 - loss: 3.0970
Epo

In [3]:
model.save("models/lstm.h5")